In [2]:
import os
import atexit
import re
import uuid
import itertools
import ast
import json
import io
import sys
from IPython.display import display, HTML
from datetime import datetime
if 'spark' not in vars():
    import findspark
    findspark.init()
    from pyspark.sql import SparkSession, DataFrame
    from pyspark.sql.window import Window
    import pyspark.sql.types as T
    import pyspark.sql.functions as F
    from pyspark.storagelevel import StorageLevel
    spark = SparkSession.builder.master("local[8,2]").config(
        'spark.jars.packages',
        'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1').config(
            'spark.driver.memory', '4g').enableHiveSupport().getOrCreate()
    spark.sparkContext.setLogLevel("DEBUG")
    atexit.register(lambda: spark.stop())

from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.window import Window
import pyspark.sql.types as T
import pyspark.sql.functions as F


def cleanse_val(val):
    return re.sub(r'([^\s\w\d])+', '', val.lower()).strip() if val else ''


def reg(spark_df, name=None):
    uniqsig = 'df_{0}'.format(cleanse_val(str(
        uuid.uuid4()))) if not name else name
    spark_df.createOrReplaceTempView(uniqsig)
    return uniqsig


def show(df, rows=5):
    display(df.limit(rows).toPandas())


DataFrame.reg = reg
DataFrame.dshow = show

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2e5e48cf-6bf2-4d58-b1aa-c20a1c5e5391;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.0.1/spark-sql-kafka-0-10_2.12-3.0.1.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1!spark-sql-

In [3]:
def log_details(df, epoch_id):
    print(
        f'Processing {datetime.now()}:{epoch_id}. A dataframe with {df.count()} records with the schema of {df.dtypes}'
    )


spark.readStream.format("kafka").\
    option("kafka.bootstrap.servers", "kafka:9092").\
    option("subscribe", "test").\
    option('kafka.group.id', 'seshu_test').\
    option("startingOffsets", "earliest").load().\
    writeStream.\
    outputMode("append").format("memory").option("truncate", True).\
    trigger(processingTime="1 seconds"). \
    foreachBatch(log_details) .\
    start().awaitTermination()

24/03/05 14:51:09 DEBUG AbstractCoordinator: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Sending Heartbeat request to coordinator 71406ea7aeb8:9092 (id: 2147482646 rack: null)
24/03/05 14:51:09 DEBUG AbstractCoordinator: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Received successful Heartbeat response
24/03/05 14:51:10 DEBUG MicroBatchExecution: Starting Trigger Calculation
24/03/05 14:51:10 DEBUG KafkaConsumer: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Pausing partitions [test-0]
24/03/05 14:51:10 DEBUG KafkaOffsetReader: Partitions assigned to consumer: [test-0].
24/03/05 14:51:10 DEBUG KafkaOffsetReader: Seeking to the end.
24/03/05 14:51:10 INFO SubscriptionState: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Seeking to LATEST offset of partition test-0
24/03/05 14:51:10 DEBUG Fetcher: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Sending ListOffsetRequest (type=ListOffsetRequest, replicaId=-1,

Processing 2024-03-05 14:51:10.040127:43. A dataframe with 10 records with the schema of [('key', 'binary'), ('value', 'binary'), ('topic', 'string'), ('partition', 'int'), ('offset', 'bigint'), ('timestamp', 'timestamp'), ('timestampType', 'int')]


24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (78, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (74, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (70, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (69, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (75, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (77, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (80, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (68, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (73, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (82, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (85, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: removing (83, 0) from stageTCMP
24/03/05 14:51:10 DEBUG ExecutorMetricsPoller: remov

Processing 2024-03-05 14:51:11.038692:44. A dataframe with 10 records with the schema of [('key', 'binary'), ('value', 'binary'), ('topic', 'string'), ('partition', 'int'), ('offset', 'bigint'), ('timestamp', 'timestamp'), ('timestampType', 'int')]


24/03/05 14:51:12 DEBUG MicroBatchExecution: Starting Trigger Calculation
24/03/05 14:51:12 DEBUG KafkaConsumer: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Pausing partitions [test-0]
24/03/05 14:51:12 DEBUG KafkaOffsetReader: Partitions assigned to consumer: [test-0].
24/03/05 14:51:12 DEBUG KafkaOffsetReader: Seeking to the end.
24/03/05 14:51:12 INFO SubscriptionState: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Seeking to LATEST offset of partition test-0
24/03/05 14:51:12 DEBUG Fetcher: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Sending ListOffsetRequest (type=ListOffsetRequest, replicaId=-1, partitionTimestamps={test-0={timestamp: -1, maxNumOffsets: 1, currentLeaderEpoch: Optional[0]}}, isolationLevel=READ_UNCOMMITTED) to broker 71406ea7aeb8:9092 (id: 1001 rack: null)
24/03/05 14:51:12 DEBUG Fetcher: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Handling ListOffsetResponse response for test-0. Fetched offset 

...

Processing 2024-03-05 14:52:24.030794:117. A dataframe with 10 records with the schema of [('key', 'binary'), ('value', 'binary'), ('topic', 'string'), ('partition', 'int'), ('offset', 'bigint'), ('timestamp', 'timestamp'), ('timestampType', 'int')]


24/03/05 14:52:24 DEBUG AbstractCoordinator: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Sending Heartbeat request to coordinator 71406ea7aeb8:9092 (id: 2147482646 rack: null)
24/03/05 14:52:24 DEBUG AbstractCoordinator: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Received successful Heartbeat response
24/03/05 14:52:25 DEBUG MicroBatchExecution: Starting Trigger Calculation
24/03/05 14:52:25 DEBUG KafkaConsumer: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Pausing partitions [test-0]
24/03/05 14:52:25 DEBUG KafkaOffsetReader: Partitions assigned to consumer: [test-0].
24/03/05 14:52:25 DEBUG KafkaOffsetReader: Seeking to the end.
24/03/05 14:52:25 INFO SubscriptionState: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Seeking to LATEST offset of partition test-0
24/03/05 14:52:25 DEBUG Fetcher: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Sending ListOffsetRequest (type=ListOffsetRequest, replicaId=-1,

Processing 2024-03-05 14:52:25.028131:118. A dataframe with 10 records with the schema of [('key', 'binary'), ('value', 'binary'), ('topic', 'string'), ('partition', 'int'), ('offset', 'bigint'), ('timestamp', 'timestamp'), ('timestampType', 'int')]


24/03/05 14:52:26 DEBUG MicroBatchExecution: Starting Trigger Calculation
24/03/05 14:52:26 DEBUG KafkaConsumer: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Pausing partitions [test-0]
24/03/05 14:52:26 DEBUG KafkaOffsetReader: Partitions assigned to consumer: [test-0].
24/03/05 14:52:26 DEBUG KafkaOffsetReader: Seeking to the end.
24/03/05 14:52:26 INFO SubscriptionState: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Seeking to LATEST offset of partition test-0
24/03/05 14:52:26 DEBUG Fetcher: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Sending ListOffsetRequest (type=ListOffsetRequest, replicaId=-1, partitionTimestamps={test-0={timestamp: -1, maxNumOffsets: 1, currentLeaderEpoch: Optional[0]}}, isolationLevel=READ_UNCOMMITTED) to broker 71406ea7aeb8:9092 (id: 1001 rack: null)
24/03/05 14:52:26 DEBUG Fetcher: [Consumer clientId=consumer-seshu_test-1, groupId=seshu_test] Handling ListOffsetResponse response for test-0. Fetched offset 

Processing 2024-03-05 14:52:26.036464:119. A dataframe with 10 records with the schema of [('key', 'binary'), ('value', 'binary'), ('topic', 'string'), ('partition', 'int'), ('offset', 'bigint'), ('timestamp', 'timestamp'), ('timestampType', 'int')]
